In [1]:
!pip install qiskit_optimization # task completed in github codespace

> Define the ILP formulation of the BPP. You can use Docplex or similar frameworks to do it. 

Given $n$ items, each with an associated weight $w_i$, and bins with maximum weight (capacity) $C$. $x_{ij}$ represent decision variables that equals 1 if item $i$ is put into bin $j$, and $y_j$ = 1 if bin $j$ is used, bounded by the maximum number of bins $n$. The objective is to minimize the number of bins
$$
\quad \sum_{j=1}^{n} y_j
$$

subject to constraints that 
1. 1 item is assigned to 1 bin
$$
\quad \sum_{j=1}^{n} x_{ij} = 1, \quad \forall i = 1, \ldots, n
$$
2. total weight of items respect the bin capacity
$$
\quad \sum_{i=1}^{n} w_i x_{ij} \leq C y_j, \quad \forall j = 1, \ldots, n
$$
3. if a bin is not used $y_j = 0$, no items should be assigned to it $x_{ij} = 0$ for all i. 
$$
\quad x_{ij} \leq y_j, \quad \forall i = 1, \ldots, n, \forall j = 1, \ldots, n
$$
The capacity constraint usually implies this constraint, but it is included to improve the performance of the model.

From Integer Linear Programming (ILP) to Quadratic Unconstrained Binary Optimization (QUBO)

Define the ILP formulation of the BPP. You can use Docplex or similar frameworks to do it. 
Create a function to transform the ILP model into a QUBO 
Test your function with specific instances (size small, medium, and big) 

Create a Brute Force solver for the QUBO problem and solve the specific instances. 

To solve the QUBO, use quantum annealing simulators. You can use the Dwave Ocean Framework. Here is an example.

Use a Quantum Variational approach to solve the QUBO. 

Create multiple Ansantz for tests. 
Build a function with input being the QUBO and Ansantz. Using a hybrid approach solved the QUBO. 

Use QAOA to solve the QUBO. 

Create from scratch a QAOA function. 

Compare and analyze the results. 

What is the difference between QAOA, Quantum Annealing, and Quantum Variational approaches with different Ansatz? 
The QUBO model has emerged as an underpinning of the quantum computing area known as quantum annealing and Fujitsu's digital annealing, and has become a subject of study in neuromorphic computing.

How do the results compare with the brute force approach? 



In [2]:
from docplex.mp.model import Model

def define_ilp(items_weight: list, bin_capacity: float) -> Model:
    """
    Define an integer linear programming model for the bin packing problem.

    Parameters
    ----------
    items_weight : list of float
        Weights of the items.
    bin_capacity : float
        naximum weight that a bin can hold.
    """
    N = len(items_weight)
    model = Model()
    x = {(i, j): model.binary_var() for i in range(N) for j in range(N)}
    y = [model.binary_var() for j in range(N)]
    model.minimize(model.sum(y[j] for j in range(N)))
    for i in range(N):
        model.add_constraint(model.sum(x[i, j] for j in range(N)) == 1)
    for j in range(N):
        model.add_constraint(model.sum(items_weight[i] * x[i, j] for i in range(N)) <= bin_capacity * y[j])
    for i in range(N):
        for j in range(N):
            model.add_constraint(x[i, j] <= y[j])
    print(f"Number of variables: {model.number_of_variables} and constraints: {model.number_of_constraints}")
    print(f"Model objective: {model.get_objective_expr()}")
    return model


> Create a function to transform the ILP model into a QUBO 

The objective function is quatradic with the form $x^T Q x$,
where x is a vector of binary decision variables and Q is a square matrix of constants. There is no constraints so we introduce quadratic penalties into the objective function as an alternative.

1. Item Assignment Constraint:
$$
x1+x2+x3 == 1, 
$$
for each item $i$, if $\sum_{j=1}^{n} x_{ij} \neq 1$, the objective function will be increased by $\lambda$.

2. Bin Capacity Constraint:
$$
\lambda \quad \sum_{j=1}^{n}  \max (0, (C y_j - \sum_{i=1}^{n} w_i x_{ij})^2)
$$
for each bin $j$, if $\sum_{i=1}^{n} w_i x_{ij} > C$, the objective function will be increased by $\lambda$. The max(0,⋅) function ensures no penalty if weight is less than capacity.

The total objective function:
$$
\quad \sum_{j=1}^{n} y_j + \lambda_1 \quad \sum_{i=1}^{n} (1 - \sum_{j=1}^{n} x_{ij})^2 + \lambda_2 \quad \sum_{j=1}^{n}  \max (0, (C y_j - \sum_{i=1}^{n} w_i x_{ij})^2) + \lambda_3 \quad \sum_{i=1}^{n} \sum_{j=1}^{n} x_{ij} y_j
$$
The matrix Q is constructed as the first n rows and columns are the $x_{ij}$ variables, and the last n rows and columns are the $y_j$ variables. The diagonal element on (i,i) are the coefficients of linear terms $x_i^2$ and the off-diagonal elements (i, j) are the coefficients of quadratic terms $x_i x_j$.

In general, to translate a linear constraint into a quadratic penalty, we can use the following formula:
$$
\lambda \quad \sum_{i=1}^{n} (1 - \sum_{j=1}^{n} x_{ij})^2 = \lambda \quad \sum_{i=1}^{n} (1 - \sum_{j=1}^{n} x_{ij}) (1 - \sum_{j=1}^{n} x_{ij}) = \lambda \quad \sum_{i=1}^{n} (1 - 2 \sum_{j=1}^{n} x_{ij} + \sum_{j=1}^{n} x_{ij} x_{ij}) = \lambda \quad \sum_{i=1}^{n} (1 - 2 \sum_{j=1}^{n} x_{ij} + \sum_{j=1}^{n} x_{ij})
$$

for 2 items and 2 bins, the Q matrix is:
$$
\begin{bmatrix}
\lambda_1 & 0 & 0 & 0 & 0 & 0 \\
0 & \lambda_1 & 0 & 0 & 0 & 0 \\
0 & 0 & \lambda_1 & 0 & 0 & 0 \\
0 & 0 & 0 & \lambda_1 & 0 & 0 \\
0 & 0 & 0 & 0 & \lambda_2 & 0 \\
0 & 0 & 0 & 0 & 0 & \lambda_2 \\
\end{bmatrix}

$$

In [12]:
from docplex.mp.linear import AbstractLinearExpr
from docplex.mp.dvar import Var
from math import isclose

def expr_to_dict(expr):
    """
    Convert a linear expression to a dictionary.

    Parameters
    ----------
    expr : docplex.mp.linear.LinearExpr
        Linear expression.
    """
    if isinstance(expr, Var):
        expr = expr + 0
    linear_terms = {}
    for var, coeff in expr.iter_terms():
        linear_terms[var.index] = coeff
    return linear_terms, expr

def subtract(dict1: dict, dict2: dict) -> dict:
    """Calculate dict1 - dict2"""
    ret = dict1.copy()
    for key, val2 in dict2.items():
        if key in dict1:
            val1 = ret[key]
            if isclose(val1, val2):
                del ret[key]
            else:
                ret[key] -= val2
        else:
            ret[key] = -val2
    return ret
    
def ilp_to_qubo(items_weight: list, bin_capacity: float, penalty_factor: float = 1.0) -> dict:
    """
    Convert an integer linear programming model to a QUBO model.

    Since standard BPP is formulated with linear objectives and constraints,
    we assume the model variables are binary, and the objective and constraints 
    are linear.

    Parameters
    ----------
    model : docplex.mp.model.Model
        Integer linear programming model.
    penalty_lambda : float
        Penalty factor for the constraints.
    """
    num_variables = len(items_weight) ** 2 + len(items_weight)
    Q = np.zeros((num_variables, num_variables))
    # objective
    for i in range(len(items_weight)):
        Q[i, i] = 1
    # constraints
    for i in range(len(items_weight)):
        Q[i, i] = 1
    for j in range(len(items_weight)):
        Q[j, j] = 1
    for i in range(len(items_weight)):
        for j in range(len(items_weight)):
            Q[i, j] = 1
    for i in range(len(items_weight)):
        for j in range(len(items_weight)):
            Q[i, j] = 1
        Q[x.index, x.index] = coeff




> Test your function with specific instances (size small, medium, and big) 

In [13]:
import numpy as np
from qiskit_optimization.translators import from_docplex_mp

np.random.seed(42)

instances = {
    'small': {
        'weights': [2, 3, 5], 
        'bin_capacity': 7  
    },
    'medium': {
        'weights': np.random.randint(1, 10, size=15),
        'bin_capacity': 20
    },
    'large': {
        'weights': np.random.randint(1, 50, size=100),
        'bin_capacity': 100                     
    }
}

model = define_ilp(instances['small']['weights'], instances['small']['bin_capacity'])
qubo = ilp_to_qubo(model, penalty_factor=1.0)
test_qubo = from_docplex_mp(model)
print(test_qubo.prettyprint())
Q = test_qubo.objective.linear.to_array()
for c in test_qubo.linear_constraints:
    Q = Q + c.linear.to_array()
print(Q)

Number of variables: 12 and constraints: 15
Model objective: x10+x11+x12
x1+x2+x3 == 1
linear_terms: {0: 1, 1: 1, 2: 1}, rhs: 1
x4+x5+x6 == 1
linear_terms: {3: 1, 4: 1, 5: 1}, rhs: 1
x7+x8+x9 == 1
linear_terms: {6: 1, 7: 1, 8: 1}, rhs: 1
2x1+3x4+5x7 <= 7x10
linear_terms: {0: 2, 3: 3, 6: 5, 9: -7}, rhs: 0
2x2+3x5+5x8 <= 7x11
linear_terms: {1: 2, 4: 3, 7: 5, 10: -7}, rhs: 0
2x3+3x6+5x9 <= 7x12
linear_terms: {2: 2, 5: 3, 8: 5, 11: -7}, rhs: 0
x1 <= x10
linear_terms: {0: 1, 9: -1}, rhs: 0
x2 <= x11
linear_terms: {1: 1, 10: -1}, rhs: 0
x3 <= x12
linear_terms: {2: 1, 11: -1}, rhs: 0
x4 <= x10
linear_terms: {3: 1, 9: -1}, rhs: 0
x5 <= x11
linear_terms: {4: 1, 10: -1}, rhs: 0
x6 <= x12
linear_terms: {5: 1, 11: -1}, rhs: 0
x7 <= x10
linear_terms: {6: 1, 9: -1}, rhs: 0
x8 <= x11
linear_terms: {7: 1, 10: -1}, rhs: 0
x9 <= x12
linear_terms: {8: 1, 11: -1}, rhs: 0
Problem name: docplex_model4

Minimize
  x10 + x11 + x9

Subject to
  Linear constraints (15)
    x0 + x1 + x2 == 1  'c0'
    x3 + x4 + 

> Create a Brute Force solver for the QUBO problem and solve the specific instances. 

1. Generate all $2^n$ possible solutions; in practice, we can limit the search space by the constraints.
2. Evaluate the objective function for each combination
3. Track the best solution

In [8]:
def qubo_brute_force(Q, items_weight = []):
    from itertools import product
    n = Q.shape[0]
    if len(items_weight) > 0:
        assert n == len(items_weight) * 2
    min_energy = float('inf')
    min_state = None
    for state in product([0, 1], repeat=n):
        energy = np.dot(state, np.dot(Q, state))
        if energy < min_energy:
            min_energy = energy
            min_state = state
    return min_state, min_energy
model = define_ilp(instances['small']['weights'], instances['small']['bin_capacity'])
qubo = ilp_to_qubo(model, penalty_factor=1.0)
qubo_brute_force(qubo)

Number of variables: 12 and constraints: 15
Model objective: x10+x11+x12
x1+x2+x3 == 1
x4+x5+x6 == 1
x7+x8+x9 == 1
2x1+3x4+5x7 <= 7x10
2x2+3x5+5x8 <= 7x11
2x3+3x6+5x9 <= 7x12
x1 <= x10
x2 <= x11
x3 <= x12
x4 <= x10
x5 <= x11
x6 <= x12
x7 <= x10
x8 <= x11
x9 <= x12
state: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), energy: 0.0
state: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1), energy: 53.0
state: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0), energy: 53.0
state: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1), energy: 106.0
state: (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0), energy: 53.0
state: (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1), energy: 106.0
state: (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0), energy: 106.0
state: (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1), energy: 159.0
state: (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0), energy: 27.0
state: (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1), energy: 8.0
state: (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0), energy: 80.0
state: (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1), energy: 61.0
state: (0, 0, 0, 0, 0, 0, 0, 0, 1, 1,

((0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), np.float64(0.0))

> To solve the QUBO, use quantum annealing simulators. You can use the Dwave Ocean Framework.

In [6]:
# !pip install dwave-ocean-sdk
# !dwave config create # config file created in github codespace


In [11]:
from dwave.system import DWaveSampler, EmbeddingComposite
import dwave.inspector

def quantum_annealing_qubo(Q, sampler):
    qubo_dict = {(i, j): Q[i, j] for i in range(Q.shape[0]) for j in range(Q.shape[1]) if Q[i, j] != 0}
    # Submit the QUBO to the quantum annealer
    sampleset = sampler.sample_qubo(qubo_dict, num_reads=100)
    best_solution = sampleset.first.sample
    best_energy = sampleset.first.energy

    return best_solution, best_energy

sampler = EmbeddingComposite(DWaveSampler())
Q_example = np.array([[1, -1, 0],
                      [-1, 2, -1],
                      [0, -1, 2]])
best_solution, best_energy = quantum_annealing_qubo(Q_example, sampler)

print("Best solution:", best_solution)
print("Best energy:", best_energy)

Best solution: {0: np.int8(0), 1: np.int8(0), 2: np.int8(0)}
Best energy: 0.0


> Use a Quantum Variational approach to solve the QUBO. 

In [ ]:
from qiskit_optimization.algorithms import Doc